ПРЕДОБРАБОТКА ДАННЫХ

In [ ]:
# загрузка необходимых библиотек
import pandas as pd 
import numpy as np 

# загрузка исходного датасета
data = pd.read_csv('ecom_yl.csv') 

# обработка названия столбцов
data.columns = [col.lower().replace(' ', '_') for col in data.columns] 

# проверка количества пропусков в столбцах
nan_counts = data.isnull().sum()

# вывод пропусков в столбцах 'region', 'device', 'channel'
rows_with_nan = data[data[['region', 'device', 'channel']].isnull().any(axis=1)]

# подсчет процента пропусков в столбцах 'region', 'device', 'channel'
for k in ['region', 'device', 'channel']:
    missing = (data[k].isnull().sum() / len(data[k])) * 100
    #print(missing)

# удаление строк с пропусками в столбцах 'region', 'device', 'channel' (тк процент всего 1.3% от общего числа)
data.dropna(subset=['region', 'device', 'channel'], inplace=True)

# заполнение пропусков в столбцах связанных с покупкой
data['revenue'].fillna(0, inplace=True) 
data.fillna('No', inplace=True)

# отбор категориальных столбцов
categorical = ['region', 'device', 'channel', 'payment_type', 'promo_code']

# вывод уникальных значений в категориальных столбцах 
for cat in categorical:
    unique = data[cat].unique()
    #print(f'{cat}: {unique}')
    #print()

# обработка неявных дубликатов
data['region'].replace({'Unjted States': 'United States',
                        'Frаnce': 'France',
                        'Frаncе': 'France',
                        'Franсe': 'France',
                        'UК': 'UK',
                        'germany': 'Germany'}, inplace=True)

data['device'].replace('android', 'Android', inplace=True)
data['channel'].replace('контексная реклама', 'контекстная реклама', inplace=True)
data = data[data['promo_code'] != 0.8627150632632511]

# перевод столбцов со временем в соответствующий формат
data['session_start'] = pd.to_datetime(data['session_start']) 
data['session_end'] = pd.to_datetime(data['session_end']) 
data['session_date'] = pd.to_datetime(data['session_date']) 
data['order_dt'] = pd.to_datetime(data['order_dt'], errors='coerce')

# создание нового столбца с суммой покупки с учетом промокода
data['final_revenue'] = data['revenue']

for index, row in data.iterrows():
    promo = row['promo_code']
    if promo == 1.0:
        data.at[index, 'final_revenue'] = row['revenue'] * 0.9

# создание функции для удаления выбросов
def outlier(df, col):
    f_b = df[col].quantile(0.25)
    s_b = df[col].quantile(0.75)
    iqr = s_b - f_b
    lower = f_b - 3 * iqr
    higher = s_b + 3 * iqr
    med = df[col].median()
    df.drop(df[(df[col] < lower) | (df[col] > higher)].index, inplace=True)
    return df

# удаление выбросов в только "платящем" датасете
data_payed = data[data['revenue'] != 0]
outlier(data_payed, 'revenue')

# объединение датасетов и фильтрация
data = pd.concat([data_payed, data[data['revenue'] == 0]])
data['revenue'].describe()

# удаление выбросов в столбце с временем сессии
outlier(data, 'sessiondurationsec')

# создание столбца с временем посещения
conditions = [ 
    (data['hour_of_day'] >= 6) & (data['hour_of_day'] < 10), 
    (data['hour_of_day'] >= 10) & (data['hour_of_day'] < 17), 
    (data['hour_of_day'] >= 17) & (data['hour_of_day'] < 22), 
    (data['hour_of_day'] >= 22) | (data['hour_of_day'] < 6) 
]

values = ['morning', 'day', 'evening', 'night'] 
data['visit_period'] = np.select(conditions, values) 
 
# создание столбца с платежными пользователями
data['payer'] = data['payment_type'] != 'No' 

# сохранение обработанного датасета
data.to_csv('processed.csv', index=False)

: 

ПРЕДОБРАБОТКА ДАННЫХ

In [ ]:
# загрузка необходимых библиотек
import pandas as pd 
import matplotlib.pyplot as plt

# загрузка датасета
df = pd.read_csv('processed.csv')

# создание функции для вывода количества платящих пользователей и доли продаж
def conver(df, col):
    table = df.groupby(col)['payer'].value_counts().unstack(fill_value=0)
    table['доля_продаж'] = table[True] / (table[True] + table[False])
    table_sorted = table.sort_values(by='доля_продаж', ascending=False)
    return table_sorted

# вывод таблиц
conver(df, 'region')
conver(df, 'channel')
conver(df, 'device')

# cоздание функции для вывода графиков сезонности продаж
def seasoned(df, col):
    groups = df.groupby([col, 'payer']).size().reset_index(name='count')
    plt.figure(figsize=(10, 6))
    for payer in [True, False]:
        payer_data = groups[groups['payer'] == payer]
        plt.plot(payer_data[col], payer_data['count'], label=f'Payer: {payer}')
    plt.title('Сезонность продаж')
    plt.xlabel(col)
    plt.ylabel('Количество пользователей')
    plt.legend() 
    plt.grid(True)
    plt.savefig(f'График сезонности: {col}')
    plt.show()

# вывод графиков
seasoned(df, 'month')
seasoned(df, 'day')
seasoned(df, 'hour_of_day')

# создание датасета только с payers
df_paid = df[df['payer'] == True]

# подсчет количества покупок по типу оплаты
payment_counts = df_paid['payment_type'].value_counts()

# вывод диаграммы количества покупок по типу оплаты
plt.figure(figsize=(8, 8))
plt.pie(payment_counts, labels=payment_counts.index, startangle=90, autopct=lambda p: '{:.0f}'.format(p * sum(payment_counts) / 100))
plt.title('Распределение типов оплаты')
plt.savefig('Распределение типов оплаты')
plt.show()

: 

ПРОВЕДЕНИЕ РАСЧЕТОВ

In [ ]:
import pandas
import pandas as pd
import numpy as np

df = pd.read_csv('processed.csv', encoding='utf-8')
pd.set_option('display.max_columns', None)


# print(df)

# средний чек
def get_mean_final_revenue(df: pandas.DataFrame):
    return df['final_revenue'].mean()


# число покупок на 1 пользователя
def get_mean_purchase_count(df: pandas.DataFrame):
    purchase_count = df['revenue'].gt(0).sum()
    users_count = len(df['user_id'].unique())
    return purchase_count / users_count


# средняя продолжительность сессии по указанной колонке
def get_mean_session_duration_sec_by_column(df: pandas.DataFrame, col: str):
    return df.groupby(col)['sessiondurationsec'].mean()


# топ-3 *выбранная колонка* (например, рекламный канал) по среднему чеку
def top_3_mean_final_revenue_by_column(df: pandas.DataFrame, col: str):
    return df.groupby(col)['final_revenue'].mean().nlargest(3)


# топ-3 месяца по среднему чеку с разбивкой по регионам
def top_3_month_by_mean_final_revenue_with_region(df: pandas.DataFrame):
    # Создали вспомогательный датасет где средние данные по каждому месяцу и региону
    average_revenue = df.groupby(['month', 'region'])['final_revenue'].mean().reset_index()
    # Список трех лучших месяцев
    top_months = average_revenue.groupby('month')['final_revenue'].mean().nlargest(3).index.tolist()
    # Финальная группировка и упорядочивание по топу месяцев
    return average_revenue.sort_values(
        by=['month', 'final_revenue'],
        ascending=[True, False]).groupby(['month', 'region'],
                                         sort=False
                                         )['final_revenue'].mean().loc[top_months]


# MAU с информацией по рекламным каналам
def mau_by_channels(df, top_3=True):
    def top3_sum_and_sort(group):
        # Возможна работа без выделения топ 3 по каждому месяцу
        if top_3:
            top3 = group.nlargest(3, 'users_count')
            summed = top3.groupby('channel')['users_count'].sum()
        else:
            summed = group.groupby('channel')['users_count'].sum()
        return summed.sort_values(ascending=False)

    # Создаем вспомогательный датасет с количеством уникальных посетителей для каждого канала
    df_mau = df.groupby(['month', 'channel'])['user_id'].nunique().reset_index(name='users_count')

    # Выбираем топ 3 (при необходимости) по каждому месяцу и сортируем
    result = df_mau.groupby('month').apply(top3_sum_and_sort)
    return result


print("Средний чек:", get_mean_final_revenue(df))
print("Среднее кол-во покупок на пользователя:", get_mean_purchase_count(df))
print('\n')
print("Средняя продолжительность сессии по рекламным каналам:\n",
      get_mean_session_duration_sec_by_column(df, 'channel'))
print('\n')
print("Средняя продолжительность сессии по типу устройства:\n", get_mean_session_duration_sec_by_column(df, 'device'))
print('\n')
print("Топ-3 рекламных канала по среднему чеку:\n", top_3_mean_final_revenue_by_column(df, 'channel'))
print('\n')
print("Топ-3 региона по среднему чеку:\n", top_3_mean_final_revenue_by_column(df, 'region'))

print("Топ-3 месяца по среднему чеку с разбивкой по регионам.:\n",
      df.groupby(['month'])['final_revenue'].mean().nlargest(3))
print('\n')

print(f"Топ-3 месяца по среднему чеку с разбивкой по регионам:\n",
      top_3_month_by_mean_final_revenue_with_region(df))
print('\n')
print(f"MAU по каждому месяцу с разбивкой по рекламным каналам:\n", mau_by_channels(df, top_3=False))
print('\n')

print(f"MAU по каждому месяцу и топ 3 канала:\n", mau_by_channels(df))
print('\n')

#функция для создания датасета по столбцу с основными характеристиками 
def stat_dataset(df, col):
    grouped_data = df.groupby(col).agg(
        количество_пользователей=('user_id', 'count'),
        количество_уникальных_пользователей=('user_id', 'nunique'),
        количество_платящих_пользователей=('payer', 'sum'),
        сумма_продаж=('final_revenue', 'sum')
    )

    grouped_data['процент_платящих'] = (
        grouped_data['количество_платящих_пользователей'] / grouped_data['количество_пользователей'] * 100
    )

    grouped_data['средний_чек'] = (
        grouped_data['сумма_продаж'] / grouped_data['количество_платящих_пользователей']
    )

    grouped_data.to_csv(f'statistics_{col}.csv', encoding='utf-8')

# применение функции и создание датасетов
stat_dataset(df, 'region')
stat_dataset(df, 'channel')
stat_dataset(df, 'device')
stat_dataset(df, 'visit_period')
stat_dataset(df, 'payment_type')

: 

ПРОВЕРКА ГИПОТЕЗ

In [ ]:
# загрузка необходимых библиотек
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# загрузка датасетов со статистикой
df_reg = pd.read_csv('statistics_region.csv')
df_chan = pd.read_csv('statistics_channel.csv')
df_dev = pd.read_csv('statistics_device.csv')
df_hour = pd.read_csv('statistics_visit_period.csv')
df_pay = pd.read_csv('statistics_payment_type.csv')
df = pd.read_csv('./processed.csv')

# создание функции для вывода графиков для оценки зависимости среднего чека от региона, канала и устройства
def avg_check_conn(df, col):
    plt.figure(figsize=(8, 6))
    plt.bar(df[col], df['средний_чек'], color='skyblue')
    plt.xlabel(col)
    plt.ylabel('Средний чек')
    plt.grid(axis='y')  
    plt.tight_layout()
    plt.show()

# создание функции для вывода графика для оценки зависимости среднего чека от времени дня
def avg_check_hour(df, col):
    plt.figure(figsize=(10, 6))
    plt.plot(df[col], df['средний_чек'], marker='o', linestyle='-')
    plt.xlabel('время дня')
    plt.ylabel('средний чек')
    plt.title('Средний чек по времени суток')
    plt.grid(True)  
    plt.xticks(df[col])
    plt.show()

# сортировка датасета с временем суток
order = ['night', 'morning', 'day', 'evening']
cat_dtype = pd.CategoricalDtype(order, ordered=True)
df_hour['visit_period'] = df_hour['visit_period'].astype(cat_dtype)
df_hour.sort_values('visit_period', inplace=True)
df_hour.reset_index(drop=True, inplace=True)

# применение всех функций для построения графиков
avg_check_conn(df_reg, 'region')
avg_check_conn(df_chan, 'channel')
avg_check_conn(df_dev, 'device')
avg_check_hour(df_hour, 'visit_period')
avg_check_hour(df_pay, 'payment_type')

regions = df['region'].unique()

import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from scipy.stats import stats

df = pd.read_csv('./processed.csv')

regions = df['region'].unique()


# Влияет ли тип устройства на количество покупок в день по каждому региону?
# Проверка: выводим по каждому региону статистику по каждому типу устройств + среднее число покупок в день
# Результат: Небольшая зависимость от типа устройства заметна в United States,
# остальные регионы не демонстрируют зависимость
def hypothesis_1():
    for region in regions:
        region_data = df[(df['region'] == region) & (df['payer'] == True)]
        order_counts = region_data.groupby(['device', 'session_date']).size().reset_index(name='order_count')
        avg_orders_per_day = order_counts.groupby('device')['order_count'].mean().reset_index()

        avg_orders_per_day[['device', 'order_count']].to_csv(f"device_to_buy_count_{region}.csv")

        plt.figure(figsize=(10, 6))
        sns.barplot(x='device', y='order_count', data=avg_orders_per_day)
        plt.title(f'Среднее число покупок в день по типу устройства (Регион: {region})')
        plt.xlabel('Тип устройства')
        plt.ylabel('Среднее число покупок')
        plt.xticks(rotation=45)
        plt.show()

        device_groups = [list(group['order_count']) for name, group in order_counts.groupby('device')]
        f_statistic, p_value = stats.kruskal(*device_groups)
        print("P-value:", p_value)


# Влияет ли тип рекламного канала на количество покупок в день по каждому региону?
# Проверка: выводим по каждому региону статистику по каждому каналу продвижения + среднее число покупок в день
# Результат: существует заметное влияние канала продвижения в каждом из регионов
def hypothesis_2():
    for region in regions:
        region_data = df[(df['region'] == region) & (df['payer'] == True)]
        order_counts = region_data.groupby(['channel', 'session_date']).size().reset_index(name='order_count')
        avg_orders_per_day = order_counts.groupby('channel')['order_count'].mean().reset_index()

        avg_orders_per_day[['channel', 'order_count']].to_csv(f"channel_to_buy_count_{region}.csv")

        plt.figure(figsize=(10, 6))
        sns.barplot(x='channel', y='order_count', data=avg_orders_per_day)
        plt.title(f'Среднее число покупок в день по каналу продвижения (Регион: {region})')
        plt.xlabel('Канал продвижения')
        plt.ylabel('Среднее число покупок')
        plt.show()

        channel_groups = [list(group['order_count']) for name, group in order_counts.groupby('channel')]
        f_statistic, p_value = stats.kruskal(*channel_groups)
        print("P-value:", p_value)


# Есть ли взаимосвязь между продолжительностью сессии с суммой покупок?
# Проверка: корреляция по Пирсону и график
# Результат: корреляция отсутствует
def hypothesis_3():
    print('Коэффицент корреляции суммы покупок с длиной сессии:',
          df['final_revenue'].corr(df['sessiondurationsec'], method='pearson'))

    sns.scatterplot(df, x='final_revenue', y='sessiondurationsec')
    plt.title(f'Корреляция длины сессии и суммы покупок')
    plt.xlabel('Длина сессии')
    plt.ylabel('Сумма покупок')
    plt.show()

    print("P-value:", stats.spearmanr(df['final_revenue'], df['sessiondurationsec']).pvalue)


# Дополнительная гипотеза: влияет ли канал привлечения на вероятность покупки на сайте?
# Проверка: столбчатая диаграмма канал - конверсия
# Результат: органический траффик самый неэффективный, из соц сетей самый эффективный
def hypothesis_4():
    grouped_data = df.groupby('channel')['payer'].agg(['sum', 'count'])
    grouped_data['conversion'] = grouped_data['sum'] / grouped_data['count']

    print(grouped_data)
    grouped_data.reset_index()[['channel', 'conversion']].to_csv(f"channel_to_buy_probability.csv")


hypothesis_1()
hypothesis_2()
hypothesis_3()
hypothesis_4()

: 

ПОСТРОЕНИЕ РЕГРЕССИИ

In [ ]:
# загрузка необходимых библиотек
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline

# загрузка датасета
df = pd.read_csv('processed.csv')

# создание датасета с факторами для регрессионной модели
df_reg = df[['region', 'channel', 'visit_period', 'payer']]

# разделение датасета на факторы
X = df_reg.drop('payer', axis=1)
y = df_reg['payer']

# разделение датасета на обучающий и тестовый
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.15, random_state=10)

# инициализация категоризатора
cat = ['region', 'channel', 'visit_period']
cat_transformer = OneHotEncoder(drop='first')

# инициализация модели
model = Pipeline(steps=[
    ('preprocessor', ColumnTransformer(transformers=[
        ('cat', cat_transformer, cat)])),
    ('classifier', LogisticRegression(class_weight='balanced')) 
])

# обучение модели
model.fit(X_train, y_train)

# предсказание
y_pred = model.predict(X_test)

# проверка предсказаний
print("Предсказанные значения:", y_pred)
print("Реальные значения:", y_test.values)
accuracy = model.score(X_test, y_test)
print("Точность модели на тестовом наборе данных:", accuracy)

: 

ВЫВОДЫ:
---
1. средний чек отличается в зависимости от времени суток? ✅ (днем(10:00-16:59) и ночью(22:00-05:59) происходят существенно больше покупок, чем утром и вечером)

2. средний чек отличается в зависимости от рекламного канала? ✅ (средний чек покупок больше после email-рассылок и рекламы у блогеров)

3. влияет ли тип рекламного канала на количество покупок в день по каждому региону? ✅ (зависимость есть, во всех регионах хорошо показывает себя контекстная реклама, лишь в США лидируют социальные сети)

4. влияет ли канал привлечения на вероятность покупки на сайте? ✅ (влияние присутствует, чаще всего покупку совершают пользователи из социальных сетей и от блогеров, а хуже всего - органичные пользователи)

5. средний чек отличается в зависимости от региона? ✅❌ (средний чек немного больше во Франции и Великобритании, однако зависимость небольшая)

6. Влияет ли тип устройства на количество покупок в день по каждому региону? ❌ (тип устройства влияет только в США)

7. Проверить гипотезу о том, что средний чек отличается в зависимости от устройства? ❌ (зависимости не наблюдается)

8. Есть ли взаимосвязь между продолжительностью сессии с суммой покупок? ❌ (корреляция: 0.0667650793008177)

Метрика по выбору канала:
---------------------------------------------------------------------
Изучены 6 графиков: за 1, 2 и 3 место канал получает 3, 2 и 1 балл соответственно

Organic: 0 2 0 1 1 3 | 7

Блоггеры: 2 0 2 1 1 1 | 7

Контекстная: 0 1 1 3 3 2 | 10

Email: 3 0 0 1 1 1 | 6

Соц. сети: 1 3 3 0 0 0 | 7

ВЫБОР КАНАЛА: Контекстная реклама

ВЫБОР РЕГИОНА: Во Франции лучше всего работает именно Контекстная реклама, а также Франция - лучший регион по среднему чеку --> ФРАНЦИЯ


ФИНАЛЬНЫЙ ВЫВОД:
---
В 2020 году выгоднее всего инвестировать во Францию и специалиазирироваться на контекстной рекламе